In [3]:
with open('checkins.dat', 'r') as inf, open('checkins.csv', 'w') as ouf:
    count = 0
    for line in inf:
        words = [elem.strip() for elem in line.split('|')]
        if len(words) == 6 and words[3] != '' and words[4] != '':
            ouf.write(','.join(words) + '\n')
            count += 1
    print(count)

396635


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('checkins.csv', header=0)
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [7]:
df.shape

(396634, 6)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396634 entries, 0 to 396633
Data columns (total 6 columns):
id            396634 non-null int64
user_id       396634 non-null int64
venue_id      396634 non-null int64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(2), int64(3), object(1)
memory usage: 18.2+ MB


In [9]:
from sklearn import cluster

In [14]:
model = cluster.MeanShift(bandwidth=0.1)
data = df[:100000].drop(['id', 'user_id', 'venue_id', 'created_at'], axis=1)
data.shape

(100000, 2)

In [15]:
data.head()

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


In [16]:
import numpy as np

In [20]:
%%time
labels = model.fit_predict(data)
print("labels:{} \n frequency:{}".format(*np.unique(labels, return_counts=True)))

labels:[   0    1    2 ... 3228 3229 3230] 
 frequency:[12506  4692  3994 ...     1     1     1]
CPU times: user 2min 9s, sys: 27.9 ms, total: 2min 9s
Wall time: 2min 9s


In [21]:
print(labels[:10])

[ 5  7 30 65  1 23  0  1  2  8]


In [24]:
model.cluster_centers_.shape

(3231, 2)

In [29]:
centers = np.array([center for center, label, freq in zip(model.cluster_centers_, 
                                                          *np.unique(labels, return_counts=True)) if freq > 15])
print(centers)

[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [  41.61853175  -88.44556818]
 [  39.2494686   -77.1821271 ]
 [  38.65877915  -76.8856871 ]]


In [30]:
centers.shape

(592, 2)

In [35]:
offices = np.array([[ 33.751277, -118.188740], 
[25.867736, -80.324116],
[51.503016, -0.075479], 
[52.378894, 4.885084],
[39.366487, 117.036146], 
[-33.868457, 151.205134]])
offices.shape

(6, 2)

In [36]:
centers_sorted = sorted(centers, 
                        key = lambda dot: min([((dot[0] - office[0]) ** 2 + (dot[1] - office[1]) ** 2) for office in offices]))
centers_sorted[:20]

[array([-33.86063043, 151.20477593]),
 array([52.37296399,  4.89231722]),
 array([ 25.84567226, -80.3188906 ]),
 array([51.50299126, -0.12553729]),
 array([  33.80987796, -118.14892381]),
 array([ 25.78581242, -80.21793804]),
 array([ 25.70534972, -80.28342874]),
 array([ 26.01009825, -80.19999059]),
 array([  33.88832534, -118.04892817]),
 array([  33.87298601, -118.36209115]),
 array([  33.97257482, -118.16837067]),
 array([ 26.13884379, -80.33434684]),
 array([  33.98393587, -118.00740497]),
 array([ 26.12086266, -80.15890668]),
 array([  33.81730643, -117.89124917]),
 array([  34.06039755, -118.24870903]),
 array([  33.67430266, -117.85878927]),
 array([ 26.20058464, -80.25071613]),
 array([  34.03548695, -118.43899772]),
 array([  34.13146015, -118.11801181])]

In [37]:
with open('banners.txt', 'w') as ouf:
    ouf.write(str(centers_sorted[0][0]) + ' ' + str(centers_sorted[0][1]))